In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sea
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import joblib


In [58]:
df = pd.read_csv('SolAtasIMC_tratado.csv')
df.head()

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75


In [59]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
 5   value   36400 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.7+ MB
None


# Preprocesado de Datos

In [60]:
tamanio = df.shape[0]

In [61]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75
...,...,...,...,...,...,...
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,57
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,57
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95,57
25479,2023-07-12 00:00:00,21.95,22.10,21.90,22.05,64


In [62]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

,date,open,high,low,close,value
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,64
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,64
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,64
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,64
25485,2023-07-12 06:00:00,21.95,22.05,21.90,22.00,64
...,...,...,...,...,...,...
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,66
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,66
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30,66
32759,2024-05-10 08:00:00,153.30,155.10,153.15,154.95,66


In [63]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

,date,open,high,low,close,value
32761,2024-05-10 10:00:00,154.35,154.5,153.45,154.10,66
32762,2024-05-10 11:00:00,154.10,154.8,153.25,154.15,66
32763,2024-05-10 12:00:00,154.15,154.3,153.25,154.15,66
32764,2024-05-10 13:00:00,154.15,155.2,153.00,155.05,66
32765,2024-05-10 14:00:00,155.05,155.4,153.10,153.30,66
...,...,...,...,...,...,...
36395,2024-10-08 20:00:00,143.35,143.9,142.35,142.95,49
36396,2024-10-08 21:00:00,142.95,144.1,142.25,143.75,49
36397,2024-10-08 22:00:00,143.75,144.5,143.35,144.50,49
36398,2024-10-08 23:00:00,144.50,144.7,144.05,144.25,49


In [64]:
df_valitest = pd.concat([df_vali, df_test], axis=0)

Numero de horas que se utilizan en la predicción

In [65]:
numhorasconst = 4

# Declaramos y Entrenamos el modelo

Función que prepara los datos

In [66]:
def preparar_datosRandomForest(df, numhoras):
    X = []
    y= []
    for i in range(0, df.shape[0] - numhoras):
        auxy = df.iloc[i + numhoras]
        y.append(auxy.close)
        aux1 = []
        for e in range(0, numhoras):
            aux = df.iloc[i + e]
            for r in range(1, aux.size):
                aux1.append(aux[r])
        X.append(aux1)
    return (X, y)

Preparamos los datos en el formato requerido por el algoritmo para su entrenamiento

In [67]:
data = preparar_datosRandomForest(df_train, numhorasconst)
X1 = data[0]
y1 = data[1]

C:\Users\raulg\AppData\Local\Temp\ipykernel_22080\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


Preparamos los datos usados para testear el algortimo en el formato requerido

In [68]:
data = preparar_datosRandomForest(df_valitest, numhorasconst)
Xvt = data[0]
yvt = data[1]

C:\Users\raulg\AppData\Local\Temp\ipykernel_22080\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


Desarrollamos un algoritmo para entrenar distintas instacias con distintos parámetros y compararlas

In [69]:
def evalRandomForest(Testrpr, predictT):
    suma = 0
    n = len(Testrpr)
    for i in range(0,n):
        suma = abs(predictT[i] - Testrpr[i])/Testrpr[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [ ]:
def train_randomForestdepth(ini, fin, Xtr, ytr, Xvtaux, yvtaux, numhoras):
    resultados = []
    posbest = 0
    best = 100
    for i in range(ini, fin):
        regr1 = RandomForestRegressor(max_depth=i, random_state=0)
        regr1.fit(Xtr, ytr)
        predictT = regr1.predict(Xvtaux)
        valor = evalRandomForest(yvtaux, predictT)
        resultados.append({'numhoras' : numhoras,'max_depth': i, 'valor': valor})
        if valor < best:
            best = valor
            posbest = i
            if valor < 0.8:
                cadena = "Modelos/random_forest_model_h" + str(numhoras) + "_d" + str(i) + ".pkl"
                joblib.dump(regr1, cadena)
    return(posbest, best, resultados)
        

In [71]:
def train_randomForest(inih, finh, inid, find):
    resultados = []
    posbest = 0
    best = 100
    for i in range(inih, finh):
        Xtrain, ytrain = preparar_datosRandomForest(df_train, i)
        Xvtaux, yvtaux = preparar_datosRandomForest(df_valitest, i)
        valores = train_randomForestdepth(inid, find, Xtrain, ytrain, Xvtaux, yvtaux, i)
        valor = valores[1]
        resultados.extend(valores[2])
        print(str(i)+" "+str(valores[0])+" "+str(valor))
        with open('OptimizaciónRandomForestIMC.txt', 'a') as archivo:
            archivo.write("Número de horas: "+str(i)+" Profundidad: "+str(valores[0])+" Valor de emp obtenido: "+str(valor) + "\n")
        if valor < best:
            best = valor
            posbest = valores[0]
    df_resultados = pd.DataFrame(resultados)
    cadena = "Dataframes/resultados_randomForest.csv"
    df_resultados.to_csv(cadena, index=False)
    return(posbest, best)

In [72]:
tuple = train_randomForest(1, 169, 1, 600)

C:\Users\raulg\AppData\Local\Temp\ipykernel_22080\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


KeyboardInterrupt: 

In [ ]:
print(tuple[0])
print(tuple[1])

9
0.7791437185511273
